# **IMPORT NECESSARY LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **LOAD DATASET**

In [3]:
real_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/Fake_news_classification/True.csv")
fake_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/Fake_news_classification/Fake.csv")

print("Real's shape: ", real_data.shape,"\nFake's shape: ", fake_data.shape) ## datasets seem to be balanced. NO imbalance

Real's shape:  (21417, 4) 
Fake's shape:  (23481, 4)


In [4]:
real_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
real_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [7]:
fake_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [8]:
## sample news from datasets
print(real_data.title[0],"\t Real")
print(fake_data.title[0],"\t Fake")

As U.S. budget fight looms, Republicans flip their fiscal script 	 Real
 Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing 	 Fake


# **LOADING LARGE PRE-TRAINED ENGLISH MODEL**

In [1]:
import spacy
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [3]:
doc = nlp("White House, Congress prepare for talks on spending, immigration")

In [21]:
# the text has been tokenized
doc

White House, Congress prepare for talks on spending, immigration

In [15]:
import inspect
inspect.getmembers(doc)

[('_', <spacy.tokens.underscore.Underscore at 0x788f1189d840>),
 ('__bytes__', <function Doc.__bytes__>),
 ('__class__', spacy.tokens.doc.Doc),
 ('__delattr__',
  <method-wrapper '__delattr__' of spacy.tokens.doc.Doc object at 0x788f18b32500>),
 ('__dir__', <function Doc.__dir__()>),
 ('__doc__',
  'Doc(Vocab vocab, words=None, spaces=None, user_data=None, *, tags=None, pos=None, morphs=None, lemmas=None, heads=None, deps=None, sent_starts=None, ents=None)\nA sequence of Token objects. Access sentences and named entities, export\n    annotations to numpy arrays, losslessly serialize to compressed binary\n    strings. The `Doc` object holds an array of `TokenC` structs. The\n    Python-level `Token` and `Span` objects are views of this array, i.e.\n    they don\'t own the data themselves.\n\n    EXAMPLE:\n        Construction 1\n        >>> doc = nlp(u\'Some text\')\n\n        Construction 2\n        >>> from spacy.tokens import Doc\n        >>> doc = Doc(nlp.vocab, words=["hello", "wor

In [17]:
# printing entities of the text
doc.ents

(White House, Congress)

In [25]:
# printing vector of the text
doc.vector

array([-2.4071772 , -0.9718855 , -3.0410235 ,  1.3607991 ,  3.9874802 ,
       -1.3973483 ,  0.09455271,  4.093764  ,  0.50695455, -1.1277709 ,
        3.0648909 ,  2.4801683 , -3.380503  ,  1.0063609 ,  0.3467154 ,
        3.4377296 ,  1.3361945 , -0.43478453, -1.2087264 , -1.313329  ,
       -0.4212754 , -2.177009  ,  0.88965994,  0.19035362,  0.8934956 ,
        0.9169209 , -2.1374145 ,  0.2027845 , -0.79333365,  0.15273461,
       -0.84746075, -0.6925428 ,  1.0108519 , -2.3662171 , -0.63453543,
       -1.9325429 , -0.40057725,  0.8969745 , -0.08952457, -0.10654727,
        1.3431938 ,  0.37993813, -0.1929359 , -1.2518274 , -1.412191  ,
        1.0021074 ,  2.2339435 , -2.6918502 , -0.10345454,  2.5046124 ,
       -2.2878892 ,  2.8449118 , -1.5046189 , -4.2767057 , -0.8310039 ,
        2.3289962 , -1.6296134 ,  0.78711283, -0.8173355 , -0.5419018 ,
       -1.4775939 , -1.2539226 ,  2.4536607 , -0.4106251 ,  3.0488863 ,
        0.67470175, -2.2863374 , -1.813942  ,  0.5026156 ,  2.85

In [20]:
# text's token lemmatizing
for token in doc:
  print(token.lemma_)

White
House
,
Congress
prepare
for
talk
on
spending
,
immigration


# **FEATURE ENGINEERING**

In [9]:
## Label Encoding
real_data['label'] = 1
fake_data['label'] = 0

In [12]:
real_data['vector'] = real_data.title.apply(lambda x: nlp(x).vector)
fake_data['vector'] = fake_data.title.apply(lambda x: nlp(x).vector)

In [74]:
real_data

,title,text,subject,date,label,vector
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"[-2.0529835, -0.569027, -3.1433282, 0.55970275..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,"[0.557869, 2.2574153, 0.1913623, -0.041898414,..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,"[-1.4035959, 0.07810003, -0.17810063, 2.512658..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,"[0.010546171, 1.6784946, -0.5903046, 0.4184245..."
...,...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1,"[-0.39947656, 1.839119, -0.33630174, 1.7395099..."
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1,"[-2.655166, -2.585305, -0.5322915, 1.7183971, ..."
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1,"[0.43744293, -0.38460714, -1.2909328, -0.12436..."
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1,"[-2.1427143, -0.02742219, -1.2982899, 0.777572..."


In [13]:
concat_data = pd.concat([real_data[['vector','label']], fake_data[['vector','label']]])
concat_data[:5]

,vector,label
0,"[-2.0529835, -0.569027, -3.1433282, 0.55970275...",1
1,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ...",1
2,"[0.557869, 2.2574153, 0.1913623, -0.041898414,...",1
3,"[-1.4035959, 0.07810003, -0.17810063, 2.512658...",1
4,"[0.010546171, 1.6784946, -0.5903046, 0.4184245...",1


In [73]:
concat_data["vector"].to_numpy()

array([array([-2.05298352e+00, -5.69027007e-01, -3.14332819e+00,  5.59702754e-01,
               2.34338737e+00,  6.11786425e-01, -5.86090982e-01,  3.75486827e+00,
              -1.39141369e+00, -4.73719072e+00,  3.73624563e+00,  1.70822728e+00,
              -2.79547262e+00,  1.69996536e+00, -5.22788227e-01,  2.64950371e+00,
               1.50680625e+00, -7.95895338e-01, -7.25800037e-01,  2.62831807e-01,
               1.59186256e+00, -4.63966280e-01,  5.17202735e-01, -2.06071448e+00,
               4.77971852e-01, -1.81196892e+00, -7.55618155e-01, -2.83631831e-01,
               3.97582710e-01,  1.01238358e+00, -9.20317233e-01, -5.43072641e-01,
               1.01419830e+00, -1.16878462e+00, -2.02463293e+00, -2.49212241e+00,
              -1.07076192e+00, -1.44277230e-01,  6.49407327e-01,  5.81057310e-01,
               1.03119183e+00,  7.17255414e-01, -1.17987728e+00,  2.12526321e-01,
              -1.71630239e+00,  4.22548177e-03, -4.83047217e-01, -2.35084891e+00,
              -5

In [72]:
concat_data["vector"].to_numpy()[0]

array([-2.05298352e+00, -5.69027007e-01, -3.14332819e+00,  5.59702754e-01,
        2.34338737e+00,  6.11786425e-01, -5.86090982e-01,  3.75486827e+00,
       -1.39141369e+00, -4.73719072e+00,  3.73624563e+00,  1.70822728e+00,
       -2.79547262e+00,  1.69996536e+00, -5.22788227e-01,  2.64950371e+00,
        1.50680625e+00, -7.95895338e-01, -7.25800037e-01,  2.62831807e-01,
        1.59186256e+00, -4.63966280e-01,  5.17202735e-01, -2.06071448e+00,
        4.77971852e-01, -1.81196892e+00, -7.55618155e-01, -2.83631831e-01,
        3.97582710e-01,  1.01238358e+00, -9.20317233e-01, -5.43072641e-01,
        1.01419830e+00, -1.16878462e+00, -2.02463293e+00, -2.49212241e+00,
       -1.07076192e+00, -1.44277230e-01,  6.49407327e-01,  5.81057310e-01,
        1.03119183e+00,  7.17255414e-01, -1.17987728e+00,  2.12526321e-01,
       -1.71630239e+00,  4.22548177e-03, -4.83047217e-01, -2.35084891e+00,
       -5.69853365e-01,  1.65974820e+00, -1.66478550e+00,  2.12221742e+00,
        1.49578738e+00, -

# **SPLITTING DATA**

In [14]:
X_train, X_test, y_train, y_test = train_test_split(concat_data.vector, concat_data.label, test_size = 0.2, random_state = 42)

In [15]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(35918,) (35918,)
(8980,) (8980,)


In [16]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [17]:
X_train

array([[-1.5993854 , -0.548297  ,  0.71646464, ..., -0.35849157,
        -0.36722535, -1.260118  ],
       [-1.0975764 ,  4.002905  , -1.5652748 , ...,  0.40516716,
        -3.099633  ,  2.5089684 ],
       [-1.5837793 , -1.0673419 ,  0.47997558, ...,  0.1183231 ,
         0.37078312, -0.35408375],
       ...,
       [-0.1884757 , -1.9024603 , -2.2186658 , ..., -0.67746043,
        -2.171252  , -0.9518237 ],
       [-1.269032  ,  2.710764  , -5.4190874 , ...,  1.2490841 ,
        -0.24373166,  4.1428823 ],
       [ 0.85069716,  1.1924593 ,  0.31168061, ..., -2.5571926 ,
        -2.1298437 ,  2.2762284 ]], dtype=float32)

In [18]:
X_test

array([[-0.10354284,  0.41933927, -0.5620885 , ...,  0.3207734 ,
        -0.02178855, -1.0953954 ],
       [-2.6222556 , -2.4855726 ,  1.8389508 , ..., -0.90803117,
         0.89121366, -0.7767481 ],
       [-1.2965555 ,  0.45058632, -0.5564325 , ...,  0.37193072,
        -2.2558415 , -0.07499044],
       ...,
       [ 1.030638  ,  1.9809959 , -0.65504736, ...,  1.1037707 ,
        -2.6422653 , -2.2342086 ],
       [-0.20087436,  2.6316805 , -1.2005376 , ...,  0.2891084 ,
        -1.5878594 ,  0.5413297 ],
       [ 0.0293718 ,  1.7145482 ,  0.31517443, ...,  0.48755127,
        -1.2352599 ,  1.5891824 ]], dtype=float32)

# **K NEAREST NEIGHBORS**

In [23]:
KNN = make_pipeline(MinMaxScaler(), KNeighborsClassifier())

In [24]:
KNN.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [25]:
y_pred = KNN.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4650
           1       1.00      0.99      0.99      4330

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



# **LOGISTIC REGRESSION**

In [33]:
log_reg = make_pipeline(MinMaxScaler(), LogisticRegression(max_iter=int(1e4)))

In [34]:
log_reg.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('logisticregression', LogisticRegression(max_iter=10000))])

In [35]:
y_pred = log_reg.predict(X_test)

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



# **DECISION TREE**

In [37]:
dec_tree = make_pipeline(MinMaxScaler(), DecisionTreeClassifier())

In [38]:
dec_tree.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [39]:
y_pred = dec_tree.predict(X_test)

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4650
           1       0.99      0.97      0.98      4330

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



# **NAIVE BAYES**

In [41]:
bayes = make_pipeline(MinMaxScaler(), MultinomialNB())

In [42]:
bayes.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('multinomialnb', MultinomialNB())])

In [43]:
y_pred = bayes.predict(X_test)

In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4650
           1       0.98      0.97      0.98      4330

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



# **SUPPORT VECTOR MACHINES**

In [45]:
svm = make_pipeline(MinMaxScaler(), SVC())

In [46]:
svm.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

In [47]:
y_pred = svm.predict(X_test)

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

